In [1]:
import serial
import numpy as np
from finished import communicator_v2
from finished import kinematika

In [2]:
serial_port = 'COM5'
ser = serial.Serial(serial_port, 115200, timeout=1)
com = communicator_v2.SerialCommunication(ser=ser)

Printer response: [b'start\n', b'echo: External Reset\n', b'Marlin 1.1.6.2\n', b'\n', b'echo: Last Updated: 2018-11-28 | Author: (Ender-3 Pro)\n', b'echo:Compiled: May 24 2019\n', b'echo: Free Memory: 10078  PlannerBufferBytes: 1232\n', b'echo:TF init fail\n']
Printer Response: [b'echo:TF init fail\n', b'ok\n', b'ok\n', b'ok\n', b'ok\n', b'echo:Soft endstops: Off  Min: X0.00 Y0.00 Z0.00  Max: X235.00 Y235.00 Z250.00\n', b'ok\n', b'ok\n', b'X:0.00 Y:0.00 Z:0.00 E:0.00 Count X:0 Y:0 Z:0\n', b'ok\n', b'ok\n', b'FIRMWARE_NAME:Marlin Creality 3D SOURCE_CODE_URL:https://github.com/MarlinFirmware/Marlin PROTOCOL_VERSION:1.0 MACHINE_TYPE:Ender-3 Pro EXTRUDER_COUNT:1 UUID:cede2a2f-41a2-4748-9b12-c55c62f367ff\n', b'Cap:EEPROM:1\n', b'Cap:AUTOREPORT_TEMP:1\n', b'Cap:PROGRESS:0\n', b'Cap:PRINT_JOB:1\n', b'Cap:AUTOLEVEL:0\n', b'Cap:Z_PROBE:0\n', b'Cap:LEVELING_DATA:0\n', b'Cap:SOFTWARE_POWER:0\n', b'Cap:TOGGLE_LIGHTS:0\n', b'Cap:CASE_LIGHT_BRIGHTNESS:0\n', b'Cap:EMERGENCY_PARSER:0\n', b'ok\n']
Comm

In [3]:
#Dejanski parametri
b = 0.071589 # m
p = 0.21215 # m
l_1 = 0.16200 # m
l_2 = 0.2525 # m

In [ ]:
-0.90894698 -0.73087339

In [4]:
zacetni_polozaj = kinematika.izracun_kotov(b=b, p=p, l_1=l_1, l_2=l_2, h=0.32, psi_x_deg=0, psi_y_deg=0)
pozicije_motorjev = kinematika.deg2steps(zacetni_polozaj)
zacetni_polozaj, pozicije_motorjev

(array([-24.65856525, -24.65856525, -24.65856525]),
 array([8.76748987, 8.76748987, 8.76748987]))

In [5]:
com.move_to_position(pozicije_motorjev, feedrate=1000, wait=True)

response: ok
Sent: G1 X8.76749 Y8.76749 Z8.76749 F1000


In [6]:
com.disable_steppers()

Steppers disabled (unlocked).


In [2]:
start_position = np.array([0,0])
target_position = np.array([1, 0])
K_p = 1
K_i = 1
K_d = 1
max_change_rate = np.deg2rad(5) #deg/s
max_tilt_angle = np.deg2rad(18) #deg

In [7]:
PID = kinematika.PID_controller(start_position=start_position, target_position=target_position, max_tilt_angle=max_tilt_angle, max_change_rate=max_change_rate, K_p=K_p, K_i=K_i, K_d=K_d)


In [8]:
new_angle = PID.calculate(current_postition=start_position, target_position=target_position, time_step=0.05)
new_angle

array([41.05,  0.  ])

In [9]:
PID.clip_angle(tilt_angle=np.array([0,0]), new_tilt_angle=new_angle, time_step=0.05)

array([0.00436332, 0.        ])

In [20]:
current_position=np.array([0.05, 0])

In [21]:
new_angle=PID.calculate(current_postition=current_position, target_position=target_position, time_step=0.05)
new_angle

array([1.285, 0.   ])